In [5]:
cd /content/AMPLify-Feedback-main.zip

[Errno 20] Not a directory: '/content/AMPLify-Feedback-main.zip'
/content


In [6]:
!unzip /content/AMPLify-Feedback-main.zip

Archive:  /content/AMPLify-Feedback-main.zip
e9d5f5d19ce87589d84c9beee06dbd9641e3075e
   creating: AMPLify-Feedback-main/
 extracting: AMPLify-Feedback-main/.gitignore  
  inflating: AMPLify-Feedback-main/DataPreprocessing.ipynb  
  inflating: AMPLify-Feedback-main/Decode_Onehot.ipynb  
  inflating: AMPLify-Feedback-main/Loop.ipynb  
  inflating: AMPLify-Feedback-main/MICPredictor.ipynb  
  inflating: AMPLify-Feedback-main/PeptideGenerator.ipynb  
   creating: AMPLify-Feedback-main/deprecated_scripts/
  inflating: AMPLify-Feedback-main/deprecated_scripts/ActivityPredictor_tf_deprecated.ipynb  
  inflating: AMPLify-Feedback-main/deprecated_scripts/ActivityPredictor_torch.ipynb  
  inflating: AMPLify-Feedback-main/deprecated_scripts/MICGenerator.ipynb  
  inflating: AMPLify-Feedback-main/deprecated_scripts/RL.ipynb  
   creating: AMPLify-Feedback-main/model_weights/
   creating: AMPLify-Feedback-main/model_weights/MICPredictor/
  inflating: AMPLify-Feedback-main/model_weights/MICPredicto

In [54]:
seq_train = np.load("/content/AMPLify-Feedback-main/processed_data/gan_train_data/group_0.npy")
seq_train = tf.one_hot(seq_train.squeeze(), depth=43)
print("Data Shape:", seq_train.shape)

Data Shape: (22264, 190, 43)


In [55]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
import numpy as np
import math
import matplotlib.pyplot as plt

In [62]:
seq_train = np.load("/content/AMPLify-Feedback-main/processed_data/test/seq_cv.npy")
state_train = np.load("/content/AMPLify-Feedback-main/processed_data/test/state_cv.npy")
label_train = np.load("/content/AMPLify-Feedback-main/processed_data/test/label_cv.npy")
seq_val = np.load("/content/AMPLify-Feedback-main/processed_data/test/seq_test.npy")
state_val = np.load("/content/AMPLify-Feedback-main/processed_data/test/state_test.npy")
label_val = np.load("/content/AMPLify-Feedback-main/processed_data/test/label_test.npy")
seq_train = tf.one_hot(seq_train.squeeze(), depth=43)
seq_val = tf.one_hot(seq_val.squeeze(), depth=43)

In [63]:
# Generator
def build_generator(seq_length, depth, latent_dim=2):
    inputs0 = layers.Input(shape=(latent_dim,), name="Input0")
    inputs1 = layers.Input(shape=(326,), name="Input1")
    x1 = layers.Dense(latent_dim, activation='tanh', name="Input1Transform")(inputs1)
    x = layers.Concatenate(name="Concat")([inputs0, x1])
    x = layers.Dense(256, input_dim=latent_dim, activation='relu', name="Dense0")(x)
    x = layers.BatchNormalization(name="BatchNorm0")(x)
    x = layers.Dense(seq_length*10, activation='tanh', name="DenseResize")(x)
    x = layers.Reshape((seq_length, 10), name="Reshape")(x)
    x = layers.GRU(256, return_sequences=True, name="GRU0")(x)
    x = layers.Dense(depth, activation="softmax", name="Output")(x)
    model = tf.keras.models.Model(inputs=[inputs0, inputs1], outputs=x)
    return model

In [64]:
# Discriminator
def build_discriminator(seq_length, depth):
    model = tf.keras.Sequential(name="discriminator")
    model.add(layers.Flatten(name="Flatten"))
    model.add(layers.Dense(512, activation='relu', name="Dense0"))
    model.add(layers.Dropout(0.3, name="Dropout"))
    model.add(layers.Dense(256, activation='relu', name="Dense1"))
    model.add(layers.Dense(1, activation='sigmoid', name="Output"))
    return model


In [65]:
# GAN
def compile_gan(generator, discriminator):
    discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    discriminator.trainable = False
    gan_input0 = layers.Input(shape=(latent_dim,))
    gan_input1 = layers.Input(shape=(326,))
    gan_output = discriminator(generator([gan_input0, gan_input1]))
    gan = tf.keras.Model([gan_input0, gan_input1], gan_output)
    gan.compile(loss='binary_crossentropy', optimizer='adam')
    return gan


In [66]:
def train_gan(generator, discriminator, gan, seq_train, state_train, epochs, batch_size, latent_dim):
    for epoch in range(epochs):
        for i in range(0, seq_train.shape[0], batch_size):
            real_sequences = seq_train[i:i + batch_size]
            state_train_batch = state_train[i:i + batch_size]
            current_batch_size = real_sequences.shape[0]

            # Generate Fake sequence
            noise = np.random.normal(0, 1, (current_batch_size, latent_dim))
            generated_sequences = generator.predict([noise, state_train_batch])

            # Labels for real and fake data
            real_labels = np.ones((current_batch_size, 1))
            fake_labels = np.zeros((current_batch_size, 1))

            # Train discriminator
            discriminator.trainable = True
            d_loss_real = discriminator.train_on_batch(real_sequences, real_labels)
            d_loss_fake = discriminator.train_on_batch(generated_sequences, fake_labels)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            discriminator.trainable = False

            # Train generator
            noise = np.random.normal(0, 1, (current_batch_size, latent_dim))
            g_loss = gan.train_on_batch([noise, state_train_batch], np.ones((current_batch_size, 1)))

            # Print the progress
            print(f"Epoch {epoch+1}/{epochs}, Batch {i//batch_size+1}, Discriminator Loss: {d_loss}, Generator Loss: {g_loss}")

In [69]:
latent_dim = 2
seq_length = seq_train.shape[1]
depth = 43

generator = build_generator(seq_length, depth, latent_dim)
discriminator = build_discriminator(seq_length,depth)
gan = compile_gan(generator, discriminator)

# Train GAN
train_gan(generator, discriminator, gan, seq_train, state_train, epochs=5, batch_size=32, latent_dim=2)


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 151ms/step
Epoch 2/5, Batch 573, Discriminator Loss: [0.01613598 1.        ], Generator Loss: 4.739812850952148
1/1 [==============================] - 0s 144ms/step
Epoch 2/5, Batch 574, Discriminator Loss: [0.02498314 0.984375  ], Generator Loss: 4.383365154266357
1/1 [==============================] - 0s 140ms/step
Epoch 2/5, Batch 575, Discriminator Loss: [0.01163344 1.        ], Generator Loss: 5.25243616104126
1/1 [==============================] - 0s 228ms/step
Epoch 2/5, Batch 576, Discriminator Loss: [0.02288877 1.        ], Generator Loss: 5.873100757598877
1/1 [==============================] - 0s 246ms/step
Epoch 2/5, Batch 577, Discriminator Loss: [0.04429879 0.984375  ], Generator Loss: 5.900247573852539
1/1 [==============================] - 0s 228ms/step
Epoch 2/5, Batch 578, Discriminator Loss: [0.12412554 0.9375    ], Generator Loss: 4.914745807647705
1/1 [=====================

In [70]:
generated_seqs = generate_sequences(generator, latent_dim, num_sequences=20)
generated_seqs_argmax = tf.math.argmax(generated_seqs, axis=2)


1/1 [==============================] - 0s 115ms/step


ValueError: ignored

In [72]:
generator.save("/content/AMPLify-Feedback-main/model_weights/PeptideGenerator_V2.keras")
discriminator.save("/content/AMPLify-Feedback-main/model_weights/PeptideDiscriminator_V2.keras")